In [1]:
# Tuning of Regularization Hyperparameter

In [2]:
# Import libraries

import numpy as np               
import matplotlib.pyplot as plt 
import tensorflow as tf          
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split 
import os                        
import cv2                     
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score

In [3]:
# Collect and preprocess data

req_height = 64
req_width = 64
channels = 3

def load_and_preprocess_data(data_dir, image_size=(req_height, req_width)):
    data = []
    labels = []

    for folder_name in os.listdir(data_dir):
        label = 1 if folder_name == 'faces' else 0
        folder_path = os.path.join(data_dir, folder_name)

        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            img = img.astype("float32") / 255.0
            data.append(img)
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

data_dir = 'dataset'
data, labels = load_and_preprocess_data(data_dir)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

Data shape: (2560, 64, 64, 3)
Labels shape: (2560,)


In [4]:
# Split the dataset

x_train, x_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
# Create a neural network model

def create_model(alpha):
    model = Sequential([
        Flatten(input_shape=(req_width, req_height, channels)),
        Dense(256, activation='sigmoid', kernel_regularizer=l2(alpha)),  # Hidden layer 1
        Dense(128, activation='sigmoid', kernel_regularizer=l2(alpha)),  # Hidden layer 2
        Dense(1, activation='sigmoid', kernel_regularizer=l2(alpha))     # Output layer (binary classification)
    ])

    # Compile the model with hinge loss or cross-entropy loss

    # a) Hinge loss
    # model.compile(optimizer='adam', loss='hinge', metrics=['accuracy'])

    # b) Cross-entropy loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [6]:
# Define the range of regularization parameters to search over
param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0, 10]}

# Create a K-fold cross-validation splitter
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create an empty list to store cross-validation results
cv_results = []

# Perform GridSearchCV manually
for train_index, val_index in kf.split(x_train):
    train_data, val_data = x_train[train_index], x_train[val_index]
    train_labels, val_labels = y_train[train_index], y_train[val_index]

    for alpha in param_grid['alpha']:
        model = create_model(alpha)
        model.fit(train_data, train_labels, epochs=10, batch_size=32)

        # Evaluate the model on the validation set
        val_predictions = model.predict(val_data)
        val_accuracy = accuracy_score(val_labels, (val_predictions >= 0.5).astype(int))
        cv_results.append({'alpha': alpha, 'accuracy': val_accuracy})

# Find the best hyperparameters based on validation accuracy
best_params = max(cv_results, key=lambda x: x['accuracy'])

# Print the best hyperparameters and corresponding accuracy
print("Best Regularization Hyperparameter Value: ", best_params['alpha'])
print("Best Accuracy: {:.2f}%".format(best_params['accuracy'] * 100))

Epoch 1/10
45/45 [==============================] - 2s 30ms/step - loss: 0.9926 - accuracy: 0.7097
Epoch 2/10
45/45 [==============================] - 2s 36ms/step - loss: 0.5221 - accuracy: 0.8800
Epoch 3/10
45/45 [==============================] - 2s 34ms/step - loss: 0.3514 - accuracy: 0.9435
Epoch 4/10
45/45 [==============================] - 2s 36ms/step - loss: 0.3059 - accuracy: 0.9518
Epoch 5/10
45/45 [==============================] - 2s 43ms/step - loss: 0.3145 - accuracy: 0.9463
Epoch 6/10
45/45 [==============================] - 2s 36ms/step - loss: 0.3168 - accuracy: 0.9505
Epoch 7/10
45/45 [==============================] - 2s 35ms/step - loss: 0.2304 - accuracy: 0.9805
Epoch 8/10
45/45 [==============================] - 2s 34ms/step - loss: 0.2081 - accuracy: 0.9812
Epoch 9/10
45/45 [==============================] - 1s 33ms/step - loss: 0.2561 - accuracy: 0.9567
Epoch 10/10
12/12 [==============================] - 0s 4ms/step
Epoch 1/10
45/45 [==========================

45/45 [==============================] - 2s 36ms/step - loss: 0.6855 - accuracy: 0.6064
Epoch 10/10
12/12 [==============================] - 0s 4ms/step
Epoch 1/10
45/45 [==============================] - 3s 40ms/step - loss: 178.5786 - accuracy: 0.6064
Epoch 2/10
45/45 [==============================] - 2s 36ms/step - loss: 29.6333 - accuracy: 0.6064
Epoch 3/10
45/45 [==============================] - 2s 39ms/step - loss: 7.2106 - accuracy: 0.6064
Epoch 4/10
45/45 [==============================] - 2s 38ms/step - loss: 1.9408 - accuracy: 0.6064
Epoch 5/10
45/45 [==============================] - 2s 38ms/step - loss: 0.9035 - accuracy: 0.6064
Epoch 6/10
45/45 [==============================] - 2s 34ms/step - loss: 0.7238 - accuracy: 0.6064
Epoch 7/10
45/45 [==============================] - 2s 34ms/step - loss: 0.6898 - accuracy: 0.6064
Epoch 8/10
45/45 [==============================] - 2s 37ms/step - loss: 0.6800 - accuracy: 0.6064
Epoch 9/10
45/45 [==============================] - 

45/45 [==============================] - 2s 35ms/step - loss: 0.2650 - accuracy: 0.9644
Epoch 8/10
45/45 [==============================] - 2s 36ms/step - loss: 0.2270 - accuracy: 0.9791
Epoch 9/10
45/45 [==============================] - 2s 34ms/step - loss: 0.1979 - accuracy: 0.9784
Epoch 10/10
12/12 [==============================] - 0s 4ms/step
Epoch 1/10
45/45 [==============================] - 2s 33ms/step - loss: 2.7932 - accuracy: 0.7176
Epoch 2/10
45/45 [==============================] - 2s 36ms/step - loss: 1.0915 - accuracy: 0.8710
Epoch 3/10
45/45 [==============================] - 2s 35ms/step - loss: 0.6958 - accuracy: 0.9191
Epoch 4/10
45/45 [==============================] - 2s 36ms/step - loss: 0.5956 - accuracy: 0.8821
Epoch 5/10
45/45 [==============================] - 2s 37ms/step - loss: 0.5247 - accuracy: 0.8975
Epoch 6/10
45/45 [==============================] - 2s 36ms/step - loss: 0.4514 - accuracy: 0.9191
Epoch 7/10
45/45 [==============================] - 2s 

45/45 [==============================] - 3s 58ms/step - loss: 0.8843 - accuracy: 0.6032
Epoch 6/10
45/45 [==============================] - 3s 62ms/step - loss: 0.7144 - accuracy: 0.6032
Epoch 7/10
45/45 [==============================] - 3s 61ms/step - loss: 0.6855 - accuracy: 0.6032
Epoch 8/10
45/45 [==============================] - 3s 59ms/step - loss: 0.6779 - accuracy: 0.6032
Epoch 9/10
45/45 [==============================] - 3s 59ms/step - loss: 0.6753 - accuracy: 0.6032
Epoch 10/10
12/12 [==============================] - 0s 34ms/step
Epoch 1/10
45/45 [==============================] - 4s 64ms/step - loss: 1776.1548 - accuracy: 0.6032
Epoch 2/10
45/45 [==============================] - 3s 57ms/step - loss: 289.4380 - accuracy: 0.6032
Epoch 3/10
45/45 [==============================] - 2s 50ms/step - loss: 65.7500 - accuracy: 0.6032
Epoch 4/10
45/45 [==============================] - 2s 55ms/step - loss: 13.1683 - accuracy: 0.6032
Epoch 5/10
45/45 [=============================